## Day32 - NLP | Word2Vec + Sentiment Analysis

In [99]:
import pandas as pd
import numpy as np
from collections import Counter

from nltk import word_tokenize, SnowballStemmer
from nltk.corpus import stopwords
import string
import re

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

import multiprocessing

from time import time

from sklearn.cluster import KMeans

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### Data

In [2]:
# Dataset : https://www.kaggle.com/kazanova/sentiment140
# col_0 = target 
# col_5 = text
df = pd.read_csv('../dataset/twitter_SA.csv', encoding = 'ISO-8859-1', header=None)
df = df[[0,5]]
df.columns = ['target','text']

# 0 = negative
# 4 = positive -> 1
df['target'] = df.target.apply(lambda x : 1 if x == 4 else x)

In [128]:
analyser = SentimentIntensityAnalyzer()
df['predicted'] = df.text.apply(lambda x : 0 if analyser.polarity_scores(x)['compound'] < 0 else 1)
len(df[df['target'] == df['predicted']])/len(df)

0.665410625

In [147]:
analyser_test = SentimentIntensityAnalyzer()

bad_sentences = ['Today was a bad day', 'TODAY WAS A BAD DAY!!!',
                  'I hate this weather', 'You\'re disturbing me']

good_sentences = ['Today was a great day', 'TODAY WAS A GREAT DAY!!',
                  'I think yuo\' re so smart', 'You made an cool project']

for sentence in bad_sentences:
    print(f"Sentence = {sentence}", end="\t")
    print(f"Score = {analyser_test.polarity_scores(sentence)}\n")

print("\n")
for sentence in good_sentences:
    print(f"Sentence = {sentence}", end="\t")
    print(f"Score = {analyser_test.polarity_scores(sentence)}\n")

Sentence = Today was a bad day	Score = {'neg': 0.467, 'neu': 0.533, 'pos': 0.0, 'compound': -0.5423}

Sentence = TODAY WAS A BAD DAY!!!	Score = {'neg': 0.522, 'neu': 0.478, 'pos': 0.0, 'compound': -0.6571}

Sentence = I hate this weather	Score = {'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'compound': -0.5719}

Sentence = You're disturbing me	Score = {'neg': 0.623, 'neu': 0.377, 'pos': 0.0, 'compound': -0.5106}



Sentence = Today was a great day	Score = {'neg': 0.0, 'neu': 0.494, 'pos': 0.506, 'compound': 0.6249}

Sentence = TODAY WAS A GREAT DAY!!	Score = {'neg': 0.0, 'neu': 0.461, 'pos': 0.539, 'compound': 0.6892}

Sentence = I think yuo' re so smart	Score = {'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'compound': 0.541}

Sentence = You made an cool project	Score = {'neg': 0.0, 'neu': 0.635, 'pos': 0.365, 'compound': 0.3182}

